# **Model Klasifikasi Data Gambar Fashion-MNIST Menggunakan CNN**

Penelitian ini akan membuat klasifikasi terhadap data gambar menggunakan metode CNN.

## Dataset

Dataset yang digunakan adalah Fashion-MNIST yang bersumber dari Keras yaitu https://keras.io/api/datasets/fashion_mnist. Dataset tersebut berupa kumpulan data gambar yang memiliki 60.000 gambar pelatihan (training) dan 10.000 gambar pengujian (testing), di mana setiap gambar abu-abu berukuran 28x28 pixel dengan label yang terdiri dari 10 kelas. Fashion-MNIST berfungsi sebagai pengganti langsung dataset MNIST asli untuk membuat tolok ukur algoritma machine learning.

In [ ]:
# Import semua library yang dibutuhkan.
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

# Peringatan untuk berhenti.
import logging
logging.getLogger('tensorflow').disabled = True

In [ ]:
# Mengambil dataset "Fashion-MNIST".
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
# Solusi terbaik untuk menormalkan nilai input adalah mengubahnya menjadi
# skala 0 hingga 1. Setiap elemen dalam dataset ini memiliki nilai pixel 0 hingga 255,
# jadi peneliti akan membuat ulang skala dari nilai-nilai tersebut.
x_train = x_train / 255.0
x_test = x_test / 255.0

In [ ]:
# Inisiasi nama kelas.
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
# Informasi jumlah kelas dan image pada data pelatihan dan data pengujian.
print("Shape of Training Image Data: " + str(x_train.shape))
print("Shape of Training Class Data: " + str(y_train.shape))
print("Shape of Test Image Data: " + str(x_test.shape))
print("Shape of Test Class Data: " + str(y_test.shape))

In [ ]:
# Visualisasi 20 gambar pertama dari data pelatihan.
plt.figure(figsize=(15,15))
for i in range(20):
    plt.subplot(4,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(x_train[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[y_train[i]])

plt.show()

In [ ]:
# Visualisasi gambar dengan nilai pixel.
index = 0
plt.figure(figsize=(20,16))
plt.imshow(x_train[index], cmap=plt.cm.binary)
plt.xlabel(class_names[y_train[index]])
plt.colorbar()

ax = plt.gca()
ax.set_xticks(np.arange(-.5, 28, 1))
ax.set_yticks(np.arange(-.5, 28, 1))
ax.set_xticklabels(np.arange(0, 29, 1))
ax.set_yticklabels(np.arange(0, 29, 1))
ax.xaxis.tick_top()

plt.show()

## Pemodelan

In [ ]:
# Mendefinisikan model menggunakan library Keras.
model = tf.keras.models.Sequential()

# Dimulai dengan layer konvolusi yang akan mengekstrak fitur dari
# gambar input dengan menggeser filter konvolusi di atas gambar input,
# kemudian menghasilkan peta fitur.
model.add(
    tf.keras.layers.Conv2D(   # Model CNN.
        filters=128,   # Banyaknya filter yang akan kita pelajari.
        kernel_size=(3, 3),   # Ukuran peta fitur yang akan ada di atas gambar.
        strides=(1, 1),   # Bagaimana peta fitur bergeser di seluruh gambar.
        padding='valid',   # Padding tidak digunakan.
        activation='relu',   # Fungsi aktivasi menggunakan ReLU.
        input_shape=(28, 28, 1)   # Bentuk pixel input yang diharapkan untuk layer ini.
    )
)

# Layer berikutnya yang ditambahkan adalah layer Maxpooling.
# Hal ini mengurangi dimensi setiap fitur dan mengurangi jumlah
# parameter model untuk belajar, sehingga mempersingkat waktu training.
model.add(
    tf.keras.layers.MaxPooling2D(
        pool_size=(2, 2),   # Ukuran fitur yang dipetakan.
        strides=(2, 2)   # Pergeserannya pada fitur.
    )
)
          
# Mnambahkan layer dropout untuk melawan overfitting dan membuat
# model mempelajari beberapa representasi dari data yang sama
# dengan menonaktifkan neuron secara acak dalam fase belajar.
model.add(
    tf.keras.layers.Dropout(
        rate=0.25   # Menonaktifkan 25% neuron secara acak.
    )
)

# Output dari layer sebelumnya adalah tensor 3D dan harus
# diubah ke vektor 1D sebelum diberikan ke dense layer.
model.add(
    tf.keras.layers.Flatten()
)

# Dense layer yang saling terhubung ditambahkan untuk
# memetakan fitur turunan ke kelas yang diperlukan.
model.add(
    tf.keras.layers.Dense(
        units=512,   # Hidden unit.
        activation='relu'   # Fungsi aktivasi menggunakan ReLU.
    )
)

# Lapisan terakhir dengan 10 output dan fungsi aktivasi Softmax.
# Fungsi aktivasi Softmax dapat digunakan untuk menghitung output
# berdasarkan probabilitas. Setiap kelas diberi probabilitas dan
# kelas dengan probabilitas maksimum adalah output dari model.
model.add(
    tf.keras.layers.Dense(
        units=10,   # Hidden unit.
        activation='softmax'   # Fungsi aktivasi menggunakan Softmax.
    )
)

# Membangun model.
model.compile(
    loss=tf.keras.losses.sparse_categorical_crossentropy,   # Loss function menggunakan Sparse Categorical Crossentropy.
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),   # Optimizer function menggunakan Adam.
    metrics=['accuracy']   # Metrik akurasi.
)

In [ ]:
# Ringkasan struktur model.
model.summary()

In [ ]:
# Visualisasi struktur model.
tf.keras.utils.plot_model(model, to_file='struktur_model_cnn.png', show_shapes=True, show_layer_names=False)

## Pelatihan dan Validasi

In [ ]:
# Menambahkan dimensi warna kosong untuk CNN.
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# Melatih CNN menggunakan data pelatihan.
history = model.fit(
    
      # Data pelatihan: Fitur (gambar) dan kelas.
      x_train, y_train,
                    
      # Jumlah sampel yang harus dikerjakan sebelum memperbarui
      # parameter model internal melalui back propagation.
      batch_size=256,

      # Jumlah epoch untuk iterasi pada seluruh data pelatihan.
      epochs=10,

      # Model akan memisahkan data pelatihan dan data validasi.
      # Dari dataset pelatihan yang ada, 80% akan digunakan untuk pelatihan
      # dan 20% akan digunakan untuk validasi untuk mengevaluasi nilai
      # loss dan nilai akurasi model di akhir setiap epoch.
      validation_split=0.2,

      verbose=1
)

## Pengujian dan Evaluasi

In [ ]:
# Melihat hasil prediksi model untuk data pengujian, kemudian
# mengevaluasi nilai loss dan nilai akurasi model di akhir setiap epoch.
predicted_classes = np.argmax(model.predict(x_test),axis=1)
print(classification_report(y_test, predicted_classes, target_names=class_names, digits=4))

In [ ]:
# Melihat data pengujian yang salah diklasifikasikan.
incorrect = np.nonzero(predicted_classes!=y_test)[0]

# Menampilkan 8 gambar pertama yang salah diklasifikasikan dari data pengujian.
plt.figure(figsize=(15, 8))
for j, incorrect in enumerate(incorrect[0:8]):
    plt.subplot(2, 4, j+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(x_test[incorrect].reshape(28, 28), cmap="Reds")
    plt.title("Predicted: {}".format(class_names[predicted_classes[incorrect]]))
    plt.xlabel("Actual: {}".format(class_names[y_test[incorrect]]))